# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
#import multiprocessing as mp
#mp.set_start_method('forkserver')
import pandas as pd
from sqlalchemy import create_engine

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report

In [2]:
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Claude\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Claude\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('ResponseCategory', engine)
df = df.drop(df.loc[df['related'] > 1, :].index, axis=0)
categories = df.columns[-36:]
X = df['message'].values
Y = df[categories]

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train, y_train)

C:\Users\Claude\Anaconda3\envs\disaster_response\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Claude\Anaconda3\envs\disaster_response\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Claude\Anaconda3\envs\disaster_response\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Claude\Anaconda3\envs\disaster_response\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.2

C:\Users\Claude\Anaconda3\envs\disaster_response\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Claude\Anaconda3\envs\disaster_response\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Claude\Anaconda3\envs\disaster_response\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_pred = pipeline.predict(X_test)

print(classification_report(y_test, y_pred, target_names= y_test.columns.tolist()))

                        precision    recall  f1-score   support

               related       0.83      0.93      0.88      5018
               request       0.84      0.39      0.53      1133
                 offer       0.00      0.00      0.00        36
           aid_related       0.76      0.52      0.62      2737
          medical_help       0.59      0.07      0.13       528
      medical_products       0.85      0.09      0.17       349
     search_and_rescue       0.54      0.04      0.07       177
              security       1.00      0.01      0.02       124
              military       0.62      0.07      0.13       217
           child_alone       0.00      0.00      0.00         0
                 water       0.90      0.28      0.43       423
                  food       0.82      0.33      0.47       740
               shelter       0.80      0.13      0.22       601
              clothing       0.82      0.09      0.16        99
                 money       0.62      

C:\Users\Claude\Anaconda3\envs\disaster_response\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Claude\Anaconda3\envs\disaster_response\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Claude\Anaconda3\envs\disaster_response\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
C:\Users\Claude\Anaconda3\envs\disaster_response\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samp

### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
print(pipeline.get_params())

{'memory': None, 'steps': [('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function tokenize at 0x0000021A651C9A60>,
        vocabulary=None)), ('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)), ('clf', MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            w

In [9]:
parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'vect__max_df': (0.5, 0.75, 1.0),
        'vect__max_features': (None, 5000),
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [10, 20],
        'clf__estimator__min_samples_split': [2, 3]
    }

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=200, return_train_score=False, n_jobs=20)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [49]:
#import multiprocessing as mp
#mp.set_start_method('forkserver')

cv.fit(X_train, y_train)

Fitting 3 folds for each of 96 candidates, totalling 288 fits
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
Pickling array (shape=(19521,), dtype=object).
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(19521,), dtype=int64).
Memmapping (shape=(36, 19521), dtype=int64) to new file C:\Users\Claude\AppData\Local\Temp\joblib_memmapping_folder_6320_3065866726\6320-2643375498016-4cc9de79c1f8476f90b6db7500b8f0eb.pkl
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(13014,), dtype=int32).
Pickling array (shape=(6507,), dtype=int32).
Pickling array (shape=(19521,), dtype=object).
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(19521,), dtype=int64).
Memmapping (shape=(36, 19521), dtype=int64) to old file C:\Users\Claude\AppData\Local\Temp\joblib_memmapping_folder_6320_3065866726\6320-2643375498016-4cc9de79c1f8476f90b6db7500b8f0eb.pkl
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(13014,), 

[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:  1.5min
Pickling array (shape=(19521,), dtype=object).
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(19521,), dtype=int64).
Memmapping (shape=(36, 19521), dtype=int64) to old file C:\Users\Claude\AppData\Local\Temp\joblib_memmapping_folder_6320_3065866726\6320-2643375498016-4cc9de79c1f8476f90b6db7500b8f0eb.pkl
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(13014,), dtype=int32).
Pickling array (shape=(6507,), dtype=int32).
Pickling array (shape=(19521,), dtype=object).
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(19521,), dtype=int64).
Memmapping (shape=(36, 19521), dtype=int64) to old file C:\Users\Claude\AppData\Local\Temp\joblib_memmapping_folder_6320_3065866726\6320-2643375498016-4cc9de79c1f8476f90b6db7500b8f0eb.pkl
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(13014,), dtype=int32).
Pickling array (shape=(6507,), dtype=int32).
Pickling array (sh

[Parallel(n_jobs=20)]: Done  15 tasks      | elapsed:  3.2min
Pickling array (shape=(19521,), dtype=object).
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(19521,), dtype=int64).
Memmapping (shape=(36, 19521), dtype=int64) to old file C:\Users\Claude\AppData\Local\Temp\joblib_memmapping_folder_6320_3065866726\6320-2643375498016-4cc9de79c1f8476f90b6db7500b8f0eb.pkl
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(13014,), dtype=int32).
Pickling array (shape=(6507,), dtype=int32).
[Parallel(n_jobs=20)]: Done  16 tasks      | elapsed:  3.3min
Pickling array (shape=(19521,), dtype=object).
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(19521,), dtype=int64).
Memmapping (shape=(36, 19521), dtype=int64) to old file C:\Users\Claude\AppData\Local\Temp\joblib_memmapping_folder_6320_3065866726\6320-2643375498016-4cc9de79c1f8476f90b6db7500b8f0eb.pkl
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(13014,), dtype=int32).
P

[Parallel(n_jobs=20)]: Done  31 tasks      | elapsed:  5.4min
Pickling array (shape=(19521,), dtype=object).
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(19521,), dtype=int64).
Memmapping (shape=(36, 19521), dtype=int64) to old file C:\Users\Claude\AppData\Local\Temp\joblib_memmapping_folder_6320_3065866726\6320-2643375498016-4cc9de79c1f8476f90b6db7500b8f0eb.pkl
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(13014,), dtype=int32).
Pickling array (shape=(6507,), dtype=int32).
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:  5.9min
Pickling array (shape=(19521,), dtype=object).
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(19521,), dtype=int64).
Memmapping (shape=(36, 19521), dtype=int64) to old file C:\Users\Claude\AppData\Local\Temp\joblib_memmapping_folder_6320_3065866726\6320-2643375498016-4cc9de79c1f8476f90b6db7500b8f0eb.pkl
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(13014,), dtype=int32).
P

[Parallel(n_jobs=20)]: Done  47 tasks      | elapsed:  7.8min
Pickling array (shape=(19521,), dtype=object).
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(19521,), dtype=int64).
Memmapping (shape=(36, 19521), dtype=int64) to old file C:\Users\Claude\AppData\Local\Temp\joblib_memmapping_folder_6320_3065866726\6320-2643375498016-4cc9de79c1f8476f90b6db7500b8f0eb.pkl
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(13014,), dtype=int32).
Pickling array (shape=(6507,), dtype=int32).
[Parallel(n_jobs=20)]: Done  48 tasks      | elapsed:  7.8min
Pickling array (shape=(19521,), dtype=object).
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(19521,), dtype=int64).
Memmapping (shape=(36, 19521), dtype=int64) to old file C:\Users\Claude\AppData\Local\Temp\joblib_memmapping_folder_6320_3065866726\6320-2643375498016-4cc9de79c1f8476f90b6db7500b8f0eb.pkl
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(13014,), dtype=int32).
P

...

[Parallel(n_jobs=20)]: Done 206 tasks      | elapsed: 35.4min
Pickling array (shape=(19521,), dtype=object).
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(19521,), dtype=int64).
Memmapping (shape=(36, 19521), dtype=int64) to old file C:\Users\Claude\AppData\Local\Temp\joblib_memmapping_folder_6320_3065866726\6320-2643375498016-4cc9de79c1f8476f90b6db7500b8f0eb.pkl
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(13014,), dtype=int32).
Pickling array (shape=(6507,), dtype=int32).
[Parallel(n_jobs=20)]: Done 207 tasks      | elapsed: 35.5min
Pickling array (shape=(19521,), dtype=object).
Pickling array (shape=(36,), dtype=object).
[Parallel(n_jobs=20)]: Done 208 tasks      | elapsed: 35.5min
Pickling array (shape=(19521,), dtype=int64).
Memmapping (shape=(36, 19521), dtype=int64) to old file C:\Users\Claude\AppData\Local\Temp\joblib_memmapping_folder_6320_3065866726\6320-2643375498016-4cc9de79c1f8476f90b6db7500b8f0eb.pkl
Pickling array (shape=(36,), 

Pickling array (shape=(19521,), dtype=object).
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(19521,), dtype=int64).
Memmapping (shape=(36, 19521), dtype=int64) to old file C:\Users\Claude\AppData\Local\Temp\joblib_memmapping_folder_6320_3065866726\6320-2643375498016-4cc9de79c1f8476f90b6db7500b8f0eb.pkl
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(13014,), dtype=int32).
Pickling array (shape=(6507,), dtype=int32).
[Parallel(n_jobs=20)]: Done 223 tasks      | elapsed: 39.5min
Pickling array (shape=(19521,), dtype=object).
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(19521,), dtype=int64).
Memmapping (shape=(36, 19521), dtype=int64) to old file C:\Users\Claude\AppData\Local\Temp\joblib_memmapping_folder_6320_3065866726\6320-2643375498016-4cc9de79c1f8476f90b6db7500b8f0eb.pkl
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(13014,), dtype=int32).
Pickling array (shape=(6507,), dtype=int32).
[Parallel(n_jobs=2

[Parallel(n_jobs=20)]: Done 238 tasks      | elapsed: 42.1min
Pickling array (shape=(19521,), dtype=object).
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(19521,), dtype=int64).
Memmapping (shape=(36, 19521), dtype=int64) to old file C:\Users\Claude\AppData\Local\Temp\joblib_memmapping_folder_6320_3065866726\6320-2643375498016-4cc9de79c1f8476f90b6db7500b8f0eb.pkl
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(13014,), dtype=int32).
Pickling array (shape=(6507,), dtype=int32).
[Parallel(n_jobs=20)]: Done 239 tasks      | elapsed: 43.3min
Pickling array (shape=(19521,), dtype=object).
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(19521,), dtype=int64).
Memmapping (shape=(36, 19521), dtype=int64) to old file C:\Users\Claude\AppData\Local\Temp\joblib_memmapping_folder_6320_3065866726\6320-2643375498016-4cc9de79c1f8476f90b6db7500b8f0eb.pkl
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(13014,), dtype=int32).
P

Pickling array (shape=(19521,), dtype=object).
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(19521,), dtype=int64).
Memmapping (shape=(36, 19521), dtype=int64) to old file C:\Users\Claude\AppData\Local\Temp\joblib_memmapping_folder_6320_3065866726\6320-2643375498016-4cc9de79c1f8476f90b6db7500b8f0eb.pkl
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(13014,), dtype=int32).
Pickling array (shape=(6507,), dtype=int32).
[Parallel(n_jobs=20)]: Done 255 out of 288 | elapsed: 46.1min remaining:  6.0min
Pickling array (shape=(19521,), dtype=object).
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(19521,), dtype=int64).
Memmapping (shape=(36, 19521), dtype=int64) to old file C:\Users\Claude\AppData\Local\Temp\joblib_memmapping_folder_6320_3065866726\6320-2643375498016-4cc9de79c1f8476f90b6db7500b8f0eb.pkl
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(13014,), dtype=int32).
Pickling array (shape=(6507,), dtype=int32).

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=20,
       param_grid={'vect__ngram_range': ((1, 1), (1, 2)), 'vect__max_df': (0.5, 0.75, 1.0), 'vect__max_features': (None, 5000), 'tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [10, 20], 'clf__estimator__min_samples_split': [2, 3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
       scoring=None, verbose=200)

In [10]:
import pickle

with open('classifer.pkl', 'wb') as f:
    pickle.dump(cv, f)

In [11]:
y_pred = cv.predict(X_test)
print(classification_report(y_test, y_pred, target_names= y_test.columns.tolist()))

NotFittedError: This GridSearchCV instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [12]:
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier(DecisionTreeClassifier())))
])

In [13]:
print(pipeline2.get_params())

{'memory': None, 'steps': [('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function tokenize at 0x0000021A651C9A60>,
        vocabulary=None)), ('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)), ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='bes

In [14]:
parameters2 = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'vect__max_df': (0.5, 0.75, 1.0),
        'vect__max_features': (None, 5000, 10000),
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [10, 20],
        'clf__estimator__base_estimator__max_depth': [2,5]
    }


In [15]:
cv2 = GridSearchCV(pipeline2, param_grid=parameters2, verbose=200, return_train_score=False, n_jobs=20)

In [16]:
cv2.fit(X_train, y_train)

C:\Users\Claude\Anaconda3\envs\disaster_response\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Fitting 3 folds for each of 144 candidates, totalling 432 fits
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
Pickling array (shape=(19521,), dtype=object).
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(19521,), dtype=int64).
Memmapping (shape=(36, 19521), dtype=int64) to new file C:\Users\Claude\AppData\Local\Temp\joblib_memmapping_folder_17176_4406886507\17176-2312501600152-6efd0b0f06264366897d68e6ab6766cf.pkl
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(13014,), dtype=int32).
Pickling array (shape=(6507,), dtype=int32).
Pickling array (shape=(19521,), dtype=object).
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(19521,), dtype=int64).
Memmapping (shape=(36, 19521), dtype=int64) to old file C:\Users\Claude\AppData\Local\Temp\joblib_memmapping_folder_17176_4406886507\17176-2312501600152-6efd0b0f06264366897d68e6ab6766cf.pkl
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(1301

Pickling array (shape=(19521,), dtype=object).
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(19521,), dtype=int64).
Memmapping (shape=(36, 19521), dtype=int64) to old file C:\Users\Claude\AppData\Local\Temp\joblib_memmapping_folder_17176_4406886507\17176-2312501600152-6efd0b0f06264366897d68e6ab6766cf.pkl
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(13014,), dtype=int32).
Pickling array (shape=(6507,), dtype=int32).
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:  1.9min
Pickling array (shape=(19521,), dtype=object).
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(19521,), dtype=int64).
Memmapping (shape=(36, 19521), dtype=int64) to old file C:\Users\Claude\AppData\Local\Temp\joblib_memmapping_folder_17176_4406886507\17176-2312501600152-6efd0b0f06264366897d68e6ab6766cf.pkl
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(13014,), dtype=int32).
Pickling array (shape=(6507,), dtype=int32).
Pickling array

[Parallel(n_jobs=20)]: Done  14 tasks      | elapsed:  3.3min
Pickling array (shape=(19521,), dtype=object).
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(19521,), dtype=int64).
Memmapping (shape=(36, 19521), dtype=int64) to old file C:\Users\Claude\AppData\Local\Temp\joblib_memmapping_folder_17176_4406886507\17176-2312501600152-6efd0b0f06264366897d68e6ab6766cf.pkl
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(13014,), dtype=int32).
Pickling array (shape=(6507,), dtype=int32).
[Parallel(n_jobs=20)]: Done  15 tasks      | elapsed:  3.3min
Pickling array (shape=(19521,), dtype=object).
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(19521,), dtype=int64).
Memmapping (shape=(36, 19521), dtype=int64) to old file C:\Users\Claude\AppData\Local\Temp\joblib_memmapping_folder_17176_4406886507\17176-2312501600152-6efd0b0f06264366897d68e6ab6766cf.pkl
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(13014,), dtype=int32

[Parallel(n_jobs=20)]: Done  30 tasks      | elapsed:  5.7min
Pickling array (shape=(19521,), dtype=object).
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(19521,), dtype=int64).
Memmapping (shape=(36, 19521), dtype=int64) to old file C:\Users\Claude\AppData\Local\Temp\joblib_memmapping_folder_17176_4406886507\17176-2312501600152-6efd0b0f06264366897d68e6ab6766cf.pkl
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(13014,), dtype=int32).
Pickling array (shape=(6507,), dtype=int32).
[Parallel(n_jobs=20)]: Done  31 tasks      | elapsed:  6.2min
Pickling array (shape=(19521,), dtype=object).
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(19521,), dtype=int64).
Memmapping (shape=(36, 19521), dtype=int64) to old file C:\Users\Claude\AppData\Local\Temp\joblib_memmapping_folder_17176_4406886507\17176-2312501600152-6efd0b0f06264366897d68e6ab6766cf.pkl
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(13014,), dtype=int32

...

[Parallel(n_jobs=20)]: Done 350 tasks      | elapsed: 68.8min
Pickling array (shape=(19521,), dtype=object).
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(19521,), dtype=int64).
Memmapping (shape=(36, 19521), dtype=int64) to old file C:\Users\Claude\AppData\Local\Temp\joblib_memmapping_folder_17176_4406886507\17176-2312501600152-6efd0b0f06264366897d68e6ab6766cf.pkl
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(13014,), dtype=int32).
Pickling array (shape=(6507,), dtype=int32).
[Parallel(n_jobs=20)]: Done 351 tasks      | elapsed: 68.9min
Pickling array (shape=(19521,), dtype=object).
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(19521,), dtype=int64).
Memmapping (shape=(36, 19521), dtype=int64) to old file C:\Users\Claude\AppData\Local\Temp\joblib_memmapping_folder_17176_4406886507\17176-2312501600152-6efd0b0f06264366897d68e6ab6766cf.pkl
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(13014,), dtype=int32

[Parallel(n_jobs=20)]: Done 366 tasks      | elapsed: 76.1min
Pickling array (shape=(19521,), dtype=object).
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(19521,), dtype=int64).
Memmapping (shape=(36, 19521), dtype=int64) to old file C:\Users\Claude\AppData\Local\Temp\joblib_memmapping_folder_17176_4406886507\17176-2312501600152-6efd0b0f06264366897d68e6ab6766cf.pkl
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(13014,), dtype=int32).
Pickling array (shape=(6507,), dtype=int32).
[Parallel(n_jobs=20)]: Done 367 tasks      | elapsed: 76.5min
Pickling array (shape=(19521,), dtype=object).
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(19521,), dtype=int64).
Memmapping (shape=(36, 19521), dtype=int64) to old file C:\Users\Claude\AppData\Local\Temp\joblib_memmapping_folder_17176_4406886507\17176-2312501600152-6efd0b0f06264366897d68e6ab6766cf.pkl
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(13014,), dtype=int32

[Parallel(n_jobs=20)]: Done 382 tasks      | elapsed: 80.9min
Pickling array (shape=(19521,), dtype=object).
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(19521,), dtype=int64).
Memmapping (shape=(36, 19521), dtype=int64) to old file C:\Users\Claude\AppData\Local\Temp\joblib_memmapping_folder_17176_4406886507\17176-2312501600152-6efd0b0f06264366897d68e6ab6766cf.pkl
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(13014,), dtype=int32).
Pickling array (shape=(6507,), dtype=int32).
[Parallel(n_jobs=20)]: Done 383 tasks      | elapsed: 81.9min
Pickling array (shape=(19521,), dtype=object).
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(19521,), dtype=int64).
Memmapping (shape=(36, 19521), dtype=int64) to old file C:\Users\Claude\AppData\Local\Temp\joblib_memmapping_folder_17176_4406886507\17176-2312501600152-6efd0b0f06264366897d68e6ab6766cf.pkl
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(13014,), dtype=int32

Pickling array (shape=(19521,), dtype=object).
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(19521,), dtype=int64).
Memmapping (shape=(36, 19521), dtype=int64) to old file C:\Users\Claude\AppData\Local\Temp\joblib_memmapping_folder_17176_4406886507\17176-2312501600152-6efd0b0f06264366897d68e6ab6766cf.pkl
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(13014,), dtype=int32).
Pickling array (shape=(6507,), dtype=int32).
[Parallel(n_jobs=20)]: Done 399 out of 432 | elapsed: 87.9min remaining:  7.3min
Pickling array (shape=(19521,), dtype=object).
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(19521,), dtype=int64).
Memmapping (shape=(36, 19521), dtype=int64) to old file C:\Users\Claude\AppData\Local\Temp\joblib_memmapping_folder_17176_4406886507\17176-2312501600152-6efd0b0f06264366897d68e6ab6766cf.pkl
Pickling array (shape=(36,), dtype=object).
Pickling array (shape=(13014,), dtype=int32).
Pickling array (shape=(6507,), dtype=int

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...'best'),
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=20,
       param_grid={'vect__ngram_range': ((1, 1), (1, 2)), 'vect__max_df': (0.5, 0.75, 1.0), 'vect__max_features': (None, 5000, 10000), 'tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [10, 20], 'clf__estimator__base_estimator__max_depth': [2, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
       scoring=None, verbose=200)

In [17]:
with open('classifer2.pkl', 'wb') as f:
    pickle.dump(cv2, f)

In [18]:
y_pred2 = cv2.predict(X_test)
print(classification_report(y_test, y_pred2, target_names= y_test.columns.tolist()))

                        precision    recall  f1-score   support

               related       0.83      0.94      0.89      5018
               request       0.74      0.57      0.65      1133
                 offer       0.20      0.03      0.05        36
           aid_related       0.77      0.59      0.67      2737
          medical_help       0.59      0.25      0.35       528
      medical_products       0.67      0.35      0.46       349
     search_and_rescue       0.59      0.23      0.33       177
              security       0.12      0.02      0.03       124
              military       0.55      0.30      0.39       217
           child_alone       0.00      0.00      0.00         0
                 water       0.76      0.63      0.69       423
                  food       0.80      0.66      0.72       740
               shelter       0.76      0.51      0.61       601
              clothing       0.77      0.36      0.49        99
                 money       0.58      

C:\Users\Claude\Anaconda3\envs\disaster_response\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Claude\Anaconda3\envs\disaster_response\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Claude\Anaconda3\envs\disaster_response\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
C:\Users\Claude\Anaconda3\envs\disaster_response\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samp

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.